# Prep Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

import warnings
warnings.filterwarnings('ignore')

drive_path = '/content/drive/MyDrive/Kuliah/Tugas Akhir/Final Project Shared Folder'
base_url = "https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?CycleBeginYear=2017"
dataset_names = ['Demographics', 'Dietary', 'Examination', 'Laboratory', 'Questionnaire']

# Ingesting Data

In [ ]:
data_path = "Dataset/Data Versioning/Combined_All_V5.csv"

df_raw = pd.read_csv(os.path.join(drive_path, data_path), index_col = 0)
df_raw = df_raw.set_index('SEQN')
df_raw.head()

,Demog1_RIAGENDR,Demog1_RIDAGEYR,Demog1_RIDRETH3,Demog1_DMDEDUC3,Demog1_DMDEDUC2,Demog1_DMDMARTL,Demog1_DMDHHSIZ,Demog1_DMDFMSIZ,Demog1_DMDHHSZA,Demog1_DMDHHSZB,...,Quest21_SLQ330,Quest21_SLD013,Quest22_SMQ020,Quest22_SMQ621,Quest22_SMQ890,Quest22_SMQ900,Quest23_SMD470,Quest24_WHD010,Quest24_WHD020,Quest24_WHD080M
SEQN,,,,,,,,,,,,,,,,,,,,,
93703.0,2.0,2.0,6.0,NaN,NaN,NaN,5.0,5.0,3.000000e+00,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704.0,1.0,2.0,3.0,NaN,NaN,NaN,4.0,4.0,2.000000e+00,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705.0,2.0,66.0,4.0,NaN,2.0,3.0,1.0,1.0,5.397605e-79,5.397605e-79,...,b'07:00',8.0,1.0,NaN,2.0,2.0,NaN,63.0,165.0,NaN
93706.0,1.0,18.0,6.0,15.0,NaN,NaN,5.0,5.0,5.397605e-79,5.397605e-79,...,b'12:00',11.5,2.0,NaN,2.0,2.0,NaN,68.0,145.0,NaN
93707.0,1.0,13.0,7.0,6.0,NaN,NaN,7.0,7.0,5.397605e-79,3.000000e+00,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN


In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Columns: 146 entries, Demog1_RIAGENDR to Quest24_WHD080M
dtypes: float64(143), object(3)
memory usage: 10.4+ MB


In [ ]:
var_mapping_path = 'Dataset/Variable Mapping.xlsx'

var_mapping = pd.read_excel(os.path.join(drive_path, var_mapping_path))
var_mapping['group'] = var_mapping['group'].replace({'Dieta2': 'Dieta1'})
var_mapping['model_var_name'] = var_mapping['group'] + "_" + var_mapping['variable']

var_mapping

,name,title,variable,desc,is_used,lifestyle,group,model_var_name
0,Demographics,Demographic Variables and Sample Weights,SEQN,Respondent Sequence Number,True,False,Demog1,Demog1_SEQN
1,Demographics,Demographic Variables and Sample Weights,SDDSRVYR,Data Release Cycle,False,False,Demog1,Demog1_SDDSRVYR
2,Demographics,Demographic Variables and Sample Weights,RIDSTATR,Interview/Examination Status,False,False,Demog1,Demog1_RIDSTATR
3,Demographics,Demographic Variables and Sample Weights,RIAGENDR,Gender,True,False,Demog1,Demog1_RIAGENDR
4,Demographics,Demographic Variables and Sample Weights,RIDAGEYR,Age In Years At Screening,True,False,Demog1,Demog1_RIDAGEYR
...,...,...,...,...,...,...,...,...
4266,Questionnaire,Weight History,WHQ200,Age When You Had Wt Loss Surgery,False,False,Quest24,Quest24_WHQ200
4267,Questionnaire,Weight History - Youth,SEQN,Respondent Sequence Number,False,False,NaN,NaN
4268,Questionnaire,Weight History - Youth,WHQ030M,How Do You Consider Your Weight,False,False,NaN,NaN
4269,Questionnaire,Weight History - Youth,WHQ500,Trying To Do About Weight,False,False,NaN,NaN


In [ ]:
model_var = pd.DataFrame({'model_var_name': df_raw.columns.tolist()})

model_var = pd.merge(model_var, var_mapping, left_on=model_var['model_var_name'].str.lower(), right_on=var_mapping['model_var_name'].str.lower(), how='outer', indicator=True).query('_merge!="right_only"')
model_var = model_var[['model_var_name_x', 'name', 'title', 'variable', 'desc', 'is_used', 'lifestyle', 'group']].rename(columns={'model_var_name_x': 'model_var_name'}).drop_duplicates()

# This Model Var Table mapping will updated if there's a feature engineering that output new column
model_var

,model_var_name,name,title,variable,desc,is_used,lifestyle,group
0,Demog1_RIAGENDR,Demographics,Demographic Variables and Sample Weights,RIAGENDR,Gender,True,False,Demog1
1,Demog1_RIDAGEYR,Demographics,Demographic Variables and Sample Weights,RIDAGEYR,Age In Years At Screening,True,False,Demog1
2,Demog1_RIDRETH3,Demographics,Demographic Variables and Sample Weights,RIDRETH3,Race/Hispanic Origin W/ Nh Asian,True,False,Demog1
3,Demog1_DMDEDUC3,Demographics,Demographic Variables and Sample Weights,DMDEDUC3,Education Level - Children/Youth 6-19,True,False,Demog1
4,Demog1_DMDEDUC2,Demographics,Demographic Variables and Sample Weights,DMDEDUC2,Education Level - Adults 20+,True,False,Demog1
...,...,...,...,...,...,...,...,...
143,Quest22_SMQ900,Questionnaire,Smoking - Cigarette Use,SMQ900,Ever Used An E-Cigarette?,True,False,Quest22
144,Quest23_SMD470,Questionnaire,Smoking - Household Smokers,SMD470,# Of People Who Smoke Inside This Home?,True,False,Quest23
145,Quest24_WHD010,Questionnaire,Weight History,WHD010,Current Self-Reported Height (Inches),True,False,Quest24
146,Quest24_WHD020,Questionnaire,Weight History,WHD020,Current Self-Reported Weight (Pounds),True,False,Quest24


In [ ]:
# demog_raw = df_raw[[col for col in df_raw.columns if "Demog" in col]]
labor_raw = df_raw[[col for col in df_raw.columns if "Labor" in col]]
# exam_raw = df_raw[[col for col in df_raw.columns if "Exam" in col]]
# quest_raw = df_raw[[col for col in df_raw.columns if "Quest" in col]]
# dieta_raw = df_raw[[col for col in df_raw.columns if "Dieta" in col]]
labor_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Labor1_LBXTC     6738 non-null   float64
 1   Labor1_LBDTCSI   6738 non-null   float64
 2   Labor2_URXVOL1   7636 non-null   float64
 3   Labor2_URDFLOW1  5841 non-null   float64
 4   Labor2_URDTIME1  5841 non-null   float64
dtypes: float64(5)
memory usage: 433.8 KB


# Data Cleaning Functions

In [ ]:
def window_mean_by_age(data, col_name, window):

  def precalculate_window_mean(data, window):
    age_map = {}
    summary = data.groupby('Demog1_RIDAGEYR').agg({col_name:['count', 'sum']})
    summary.columns = ['count', 'sum']
    summary = summary.reset_index()

    for age in summary['Demog1_RIDAGEYR'].drop_duplicates():
      age_map[age] = np.sum(
          summary.loc[np.logical_and(summary['Demog1_RIDAGEYR'] >= age-window, summary['Demog1_RIDAGEYR'] <= age+window), 'sum']
      ) / np.sum(
          summary.loc[np.logical_and(summary['Demog1_RIDAGEYR'] >= age-window, summary['Demog1_RIDAGEYR'] <= age+window), 'count']
      )
    return age_map

  age_map = precalculate_window_mean(data, window)
  ret = []
  for idx, row in data.iterrows():
    ret.append(age_map[row['Demog1_RIDAGEYR']])
  return ret


# Cleaning Demographic Data

In [ ]:
def Demog1_DMDEDUC3(x):
  if x in [0,1,2,3,4,5,6,7,8,55,66]:
    return 1
  if x in [9,10,11,12]:
    return 2
  if x in [13,14]:
    return 3
  if x in [15]:
    return 4
  if x in [77]:
    return 7
  if x in [99]:
    return 9

demog = df_raw[[col for col in df_raw.columns if "Demog" in col]]

# Mapping education level for < 19 y.o to 20+ y.o education level
demog['Demog1_DMDEDUC3'] = demog['Demog1_DMDEDUC3'].apply(Demog1_DMDEDUC3)
demog['Demog1_DMDEDUC'] = demog['Demog1_DMDEDUC2'].combine_first(demog['Demog1_DMDEDUC3'])
demog = demog.drop(columns=['Demog1_DMDEDUC2', 'Demog1_DMDEDUC3'])
# The remaining null values on education level has age mean of 2 and max age of 9, set these with less than 9th grade
demog.loc[demog['Demog1_DMDEDUC'].isnull(), 'Demog1_DMDEDUC'] = 1

model_var = pd.concat([model_var, pd.DataFrame({
    'model_var_name': ['Demog1_DMDEDUC'],
    'name': ['Demographic'],
    'title': ['Demographic Variables and Sample Weights'],
    'variable': ['DMDEDUC'],
    'desc': ['Education Level all age'],
    'is_used': [True],
    'lifestyle':[False],
    'group': ['Demog1']
    })], ignore_index=True)

# Set Marital Status for < 19 y.o to never married
demog.loc[demog['Demog1_RIDAGEYR'] < 20, 'Demog1_DMDMARTL'] = 5

# Get average income from household income & family income, and set the null value with "Don't Know"
demog['Demog1_INDHHIN2'] = demog['Demog1_INDHHIN2'].combine_first(demog['Demog1_INDFMIN2'])
demog['Demog1_INDFMIN2'] = demog['Demog1_INDFMIN2'].combine_first(demog['Demog1_INDHHIN2'])
demog['Demog1_INDIN2'] = (demog['Demog1_INDHHIN2'] + demog['Demog1_INDFMIN2']) / 2
demog = demog.drop(columns=['Demog1_INDHHIN2', 'Demog1_INDFMIN2'])
demog.loc[demog['Demog1_INDIN2'].isnull(), 'Demog1_INDIN2'] = 99

demog = demog.drop(columns=['Demog1_INDFMPIR'])

model_var = pd.concat([model_var, pd.DataFrame({
    'model_var_name': ['Demog1_INDIN2'],
    'name': ['Demographic'],
    'title': ['Demographic Variables and Sample Weights'],
    'variable': ['INDIN2'],
    'desc': ['Average income from household income & family income'],
    'is_used': [True],
    'lifestyle':[False],
    'group': ['Demog1']
    })], ignore_index=True)

demog.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Demog1_RIAGENDR  9254 non-null   float64
 1   Demog1_RIDAGEYR  9254 non-null   float64
 2   Demog1_RIDRETH3  9254 non-null   float64
 3   Demog1_DMDMARTL  9254 non-null   float64
 4   Demog1_DMDHHSIZ  9254 non-null   float64
 5   Demog1_DMDFMSIZ  9254 non-null   float64
 6   Demog1_DMDHHSZA  9254 non-null   float64
 7   Demog1_DMDHHSZB  9254 non-null   float64
 8   Demog1_DMDHHSZE  9254 non-null   float64
 9   Demog1_DMDEDUC   9254 non-null   float64
 10  Demog1_INDIN2    9254 non-null   float64
dtypes: float64(11)
memory usage: 867.6 KB


# Cleaning Laboratory Data

In [ ]:
labor = df_raw[[col for col in df_raw.columns if "Labor" in col]]
labor

,Labor1_LBXTC,Labor1_LBDTCSI,Labor2_URXVOL1,Labor2_URDFLOW1,Labor2_URDTIME1
SEQN,,,,,
93703.0,NaN,NaN,NaN,NaN,NaN
93704.0,NaN,NaN,NaN,NaN,NaN
93705.0,157.0,4.06,248.0,1.204,206.0
93706.0,148.0,3.83,349.0,NaN,NaN
93707.0,189.0,4.89,100.0,0.526,190.0
...,...,...,...,...,...
102952.0,119.0,3.08,63.0,0.525,120.0
102953.0,182.0,4.71,63.0,0.529,119.0
102954.0,172.0,4.45,158.0,1.039,152.0


In [ ]:
# Laboratory Data

labor = df_raw[[col for col in df_raw.columns if "Labor" in col]]
col_to_fill = labor.columns

labor = labor.join(demog)

# Cleaning Cholesterol
# Participant age < 6 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 6].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group.

# Cleaning Urine Flow & Time
# Participant age < 6 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 6].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group. (age group to be determined)

# Cleaning Urine Volume
# Participant age < 3 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 3].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group. (age group to be determined)

# Option 1 Using mean for each age
# for col in col_to_fill:
#   labor[col] = labor[col].fillna(labor.groupby('Demog1_RIDAGEYR')[col].transform('mean'))

# labor = labor[labor.columns.drop(list(labor.filter(regex='Demog')))]

# Option 2 Using windowed mean based on age
labor_mean = labor.copy()

for col in col_to_fill:
  labor_mean[col] = window_mean_by_age(labor_mean[[col, 'Demog1_RIDAGEYR']], col_name=col, window=5)

labor = labor.fillna(labor_mean)
labor = labor.fillna(0)
labor = labor[labor.columns.drop(list(labor.filter(regex='Demog')))]

labor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Labor1_LBXTC     9254 non-null   float64
 1   Labor1_LBDTCSI   9254 non-null   float64
 2   Labor2_URXVOL1   9254 non-null   float64
 3   Labor2_URDFLOW1  9254 non-null   float64
 4   Labor2_URDTIME1  9254 non-null   float64
dtypes: float64(5)
memory usage: 691.8 KB


# Cleaning Examination Data

In [ ]:
# Examination Data

exam = df_raw[[col for col in df_raw.columns if "Exam" in col]]
col_to_fill = exam.columns

exam = exam.join(demog)

# BPAARM --> might need to discard, desc: "Arm selected"

# Blood Pressure Data (BPXPLS, BPXPTY, BPXSY1, BPXDI1, BPXSY2, BPXDI2, BPXSY3, BPXDI3)
# Users with age < 8 have null data (ref: exam_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 8].info())

# Body measure data
# Might need to use average for each age group or average of data (Don't use too long data window) of the users age.

for col in col_to_fill:
  exam[col] = exam[col].fillna(exam.groupby('Demog1_RIDAGEYR')[col].transform('mean'))

# ============================
# Cleaning from RB
# ============================

# ----------------------------
# Combine 1st, 2nd, and 3rd reading of blood presure in average
# ----------------------------

exam['Exami1_SysPulse'] = (exam['Exami1_BPXSY1'] + exam['Exami1_BPXSY2'] + exam['Exami1_BPXSY3'])/3
exam['Exami1_DiaPulse'] = (exam['Exami1_BPXDI1'] + exam['Exami1_BPXDI2'] + exam['Exami1_BPXDI3'])/3


exam.drop(['Exami1_BPXSY1', 'Exami1_BPXSY2', 'Exami1_BPXSY3', 'Exami1_BPXDI1', 'Exami1_BPXDI2', 'Exami1_BPXDI3'], axis=1, inplace=True)

exam['Exami1_SysPulse'] = exam['Exami1_SysPulse'].fillna(exam.groupby('Demog1_RIDAGEYR')['Exami1_SysPulse'].transform('mean'))
exam['Exami1_SysPulse'] = exam['Exami1_DiaPulse'].fillna(exam.groupby('Demog1_RIDAGEYR')['Exami1_DiaPulse'].transform('mean'))

# ----------------------------
# Fill Systolic Null Values
# ----------------------------

# fill infant (1 year and below)
# male
male_condition = (exam['Demog1_RIDAGEYR'] <= 1.0) & (exam['Demog1_RIAGENDR'] == 1)
exam.loc[exam['Exami1_SysPulse'].isna() & male_condition, 'Exami1_SysPulse'] = 85.0

# female
male_condition = (exam['Demog1_RIDAGEYR'] <= 1.0) & (exam['Demog1_RIAGENDR'] == 2)
exam.loc[exam['Exami1_SysPulse'].isna() & male_condition, 'Exami1_SysPulse'] = 86.0

# fill infant (>=2 years, <7 years)
# male
male_condition = (exam['Demog1_RIDAGEYR'] >= 2.0) & (exam['Demog1_RIDAGEYR'] < 7.0) & (exam['Demog1_RIAGENDR'] == 1)
exam.loc[exam['Exami1_SysPulse'].isna() & male_condition, 'Exami1_SysPulse'] = 88.0

# female
male_condition = (exam['Demog1_RIDAGEYR'] >= 2.0) & (exam['Demog1_RIDAGEYR'] < 7.0) & (exam['Demog1_RIAGENDR'] == 2)
exam.loc[exam['Exami1_SysPulse'].isna() & male_condition, 'Exami1_SysPulse'] = 86.0

# fill infant (>=7 years)
# male
male_condition = (exam['Demog1_RIDAGEYR'] >= 7.0) & (exam['Demog1_RIAGENDR'] == 1)
exam.loc[exam['Exami1_SysPulse'].isna() & male_condition, 'Exami1_SysPulse'] = 97.0

# female
male_condition = (exam['Demog1_RIDAGEYR'] >= 7.0) & (exam['Demog1_RIAGENDR'] == 2)
exam.loc[exam['Exami1_SysPulse'].isna() & male_condition, 'Exami1_SysPulse'] = 96.0

model_var = pd.concat([model_var, pd.DataFrame({
    'model_var_name': ['Exami1_SysPulse'],
    'name': ['Examination'],
    'title': ['Blood Pressure'],
    'variable': ['SysPulse'],
    'desc': ['Systolic Pulse Combined'],
    'is_used': [True],
    'lifestyle':[False],
    'group': ['Exami1']
    })], ignore_index=True)

# ----------------------------
# Fill Diastolic Null Values
# ----------------------------

# fill infant (1 year and below)
# male
male_condition = (exam['Demog1_RIDAGEYR'] <= 1.0) & (exam['Demog1_RIAGENDR'] == 1)
exam.loc[exam['Exami1_DiaPulse'].isna() & male_condition, 'Exami1_DiaPulse'] = 37.0

# female
female_condition = (exam['Demog1_RIDAGEYR'] <= 1.0) & (exam['Demog1_RIAGENDR'] == 2)
exam.loc[exam['Exami1_DiaPulse'].isna() & female_condition, 'Exami1_DiaPulse'] = 46.0

# fill infant (>=2 years, <7 years)
# male
male_condition = (exam['Demog1_RIDAGEYR'] >= 2.0) & (exam['Demog1_RIDAGEYR'] < 7.0) & (exam['Demog1_RIAGENDR'] == 1)
exam.loc[exam['Exami1_DiaPulse'].isna() & male_condition, 'Exami1_DiaPulse'] = 42.0

# female
female_condition = (exam['Demog1_RIDAGEYR'] >= 2.0) & (exam['Demog1_RIDAGEYR'] < 7.0) & (exam['Demog1_RIAGENDR'] == 2)
exam.loc[exam['Exami1_DiaPulse'].isna() & female_condition, 'Exami1_DiaPulse'] = 40.0

# fill infant (>=7 years)
# male
male_condition = (exam['Demog1_RIDAGEYR'] >= 7.0) & (exam['Demog1_RIAGENDR'] == 1)
exam.loc[exam['Exami1_DiaPulse'].isna() & male_condition, 'Exami1_DiaPulse'] = 57.0

# female
female_condition = (exam['Demog1_RIDAGEYR'] >= 7.0) & (exam['Demog1_RIAGENDR'] == 2)
exam.loc[exam['Exami1_DiaPulse'].isna() & female_condition, 'Exami1_DiaPulse'] = 57.0

model_var = pd.concat([model_var, pd.DataFrame({
    'model_var_name': ['Exami1_DiaPulse'],
    'name': ['Examination'],
    'title': ['Blood Pressure'],
    'variable': ['DiaPulse'],
    'desc': ['Diastolic Pulse Combined'],
    'is_used': [True],
    'lifestyle':[False],
    'group': ['Exami1']
    })], ignore_index=True)


# ----------------------------
# Fill 60 Sec Pulse
# ----------------------------

age_conditions = (exam['Demog1_RIDAGEYR'] <= 7.0)
exam.loc[exam['Exami1_BPXPLS'].isna() & age_conditions, 'Exami1_BPXPLS'] = 85.0


# ----------------------------
# Fill Height and BMI
# ----------------------------

#  Fill Height Male and female (5-7 yrs)
age_conditions = (exam['Demog1_RIDAGEYR'] >= 5.0) & (exam['Demog1_RIDAGEYR'] <= 7.0) & (exam['Demog1_RIAGENDR'] == 1.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 115.5

age_conditions = (exam['Demog1_RIDAGEYR'] >= 5.0) & (exam['Demog1_RIDAGEYR'] <= 7.0) & (exam['Demog1_RIAGENDR'] == 2.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 115.5

#  Fill Height Male and female (3-4 yrs)
age_conditions = (exam['Demog1_RIDAGEYR'] >= 3.0) & (exam['Demog1_RIDAGEYR'] < 5.0) & (exam['Demog1_RIAGENDR'] == 1.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 101.6

age_conditions = (exam['Demog1_RIDAGEYR'] >= 3.0) & (exam['Demog1_RIDAGEYR'] < 5.0) & (exam['Demog1_RIAGENDR'] == 2.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 101.6

#  Fill Height Male and female (1-2 yrs)
age_conditions = (exam['Demog1_RIDAGEYR'] >= 1.0) & (exam['Demog1_RIDAGEYR'] < 3.0) & (exam['Demog1_RIAGENDR'] == 1.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 81.28

age_conditions = (exam['Demog1_RIDAGEYR'] >= 1.0) & (exam['Demog1_RIDAGEYR'] < 3.0) & (exam['Demog1_RIAGENDR'] == 2.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 78.74

#  Fill Height Male and female (1 year and below)
age_conditions = (exam['Demog1_RIDAGEYR'] < 1.0) & (exam['Demog1_RIAGENDR'] == 1.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 71.12

age_conditions = (exam['Demog1_RIDAGEYR'] < 1.0) & (exam['Demog1_RIAGENDR'] == 2.0)
exam.loc[exam['Exami2_BMXHT'].isna() & age_conditions, 'Exami2_BMXHT'] = 68.58

# Fill BMI column
null_bmi = exam[exam['Exami2_BMXBMI'].isna()]
null_bmi['Exami2_BMXBMI'] = null_bmi['Exami2_BMXWT']/((null_bmi['Exami2_BMXHT']/100)**2)

# exam['Exami2_BMXBMI'] = exam['Exami2_BMXWT']/((exam['Exami2_BMXHT']/100)**2)
exam.loc[exam['Exami2_BMXBMI'].isna(), 'Exami2_BMXBMI'] = null_bmi['Exami2_BMXBMI']


# ----------------------------
# Drop Unnecessary Column
# ----------------------------

exam.drop(['Exami1_BPAARM','Exami2_BMXARML', 'Exami2_BMXARMC', 'Exami2_BMXLEG', 'Exami2_BMXWAIST', 'Exami2_BMXHIP', 'Exami1_BPXPTY'], axis=1, inplace=True)

exam = exam[exam.columns.drop(list(exam.filter(regex='Demog')))]

exam.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Exami1_BPXPLS    9254 non-null   float64
 1   Exami1_BPXPULS   9254 non-null   float64
 2   Exami2_BMXWT     9254 non-null   float64
 3   Exami2_BMXHT     9254 non-null   float64
 4   Exami2_BMXBMI    9254 non-null   float64
 5   Exami1_SysPulse  9254 non-null   float64
 6   Exami1_DiaPulse  9254 non-null   float64
dtypes: float64(7)
memory usage: 836.4 KB


# Cleaning Questionnaire Data

In [ ]:
quest = df_raw[[col for col in df_raw.columns if "Quest" in col]]

quest = quest.join(demog)

group_to_drop = [
    'Quest5', # Consumer Behavior Phone Follow-up Module - Adult Questions. Question is too wild, might not be applicable in app, example question: Buy food from fast food/pizza places, Notice calories at fast food/pizza place, Eat at all-you-can-eat buffet , Notice calories at recreational places, etc.
    'Quest13', # Hospital Utilization & Access to Care Questions, only have 1 col to use (How long since last healthcare visit) which have 86% null
    'Quest24', # Weight History Questions, can be derived from weight data from examination.
]

cols_to_drop = ['Quest1_ALQ121', # Past 12 mo how often have alcohol drink, might need to bring back
                'Quest2_BPD035', # Age told had hypertension, data is continuous and only have value if the Quest2_BPQ020 is yes. [Inconsistent Context when Fill NaN] How to clean without giving the wrong context to the model? e.g. if null value fill with 0, then it could mean they had hypertension at age 0?
                'Quest2_BPQ040A', # Taking prescription for hypertension, might be duplicate context as Quest2_BPQ020
                'Quest2_BPQ060', # Ever had blood cholesterol checked, might need to bring back
                'Quest2_BPQ070', # When blood cholesterol last checked, Inconsistent Context when Fill NaN
                'Quest2_BPQ100D', # Now taking prescribed medicine, lot of nulls, might be duplicate context as Quest2_BPQ020
                'Quest7_DIQ175B', # Overweight, lot of nulls, can be derived from weight on examination data
                'Quest7_DIQ175C', # Age, lot of nulls, can be get from weight on examination data
                'Quest7_DIQ175G', # Lack of physical activity, ~96% null.
                'Quest7_DIQ175H', # High blood pressure, ~96% null, might be duplicated context as Quest2_BPQ020
                'Quest7_DIQ175I', # High blood sugar, 99% null
                'Quest7_DIQ175J', # High cholesterol, 98% null
                'Quest7_DIQ175N', # Blurred vision, 98% null
                'Quest7_DIQ175T', # Frequent urination, 98% null
                'Quest7_DIQ175V', # Craving for sweet/eating a lot of sugar, 99% null
                'Quest14_INQ150', # Income from other sources, might need to bring back
                'Quest18_OCQ260', # Description of job/work situation, values are too wild, might not be effective & applicable in the application (example values: A federal government employee, A state government employee, Working without pay in family business or farm, etc.)
                'Quest22_SMQ621', # Cigarettes smoked in entire life, 87% null, same context as other cigarettes question (or not?)
                'Quest18_OCQ180' # Hours worked last week at all jobs, Dropped in RB's Notebook. if want to fill--> {'name': 'Quest18_OCQ180', 'replace': [77777, 99999]},
              ]
timestamp_cols = [
    {'name': 'Quest21_SLQ300', 'replace': ['77777', '99999']}, # "Usual sleep time on weekdays or workdays" "b'HH:MM'"
    {'name': 'Quest21_SLQ320', 'replace': ['77777', '99999']}, # Usual sleep time on weekends
    {'name': 'Quest21_SLQ330', 'replace': ['77777', '99999']}, # Usual wake time on weekends
]
cont_fill_mean = [
    {'name': 'Quest4_CBD111', 'replace': [777777, 999999]}, # Money spent on food at other stores
    {'name': 'Quest4_CBD121', 'replace': [777777, 999999]}, # Money spent on eating out
    {'name': 'Quest6_DED120', 'replace': [3333, 7777, 9999, '.']}, # Minutes outdoors 9am - 5pm work day
    {'name': 'Quest6_DED125', 'replace': [3333, 7777, 9999]}, # Minutes outdoors 9am - 5pm not work day
    {'name': 'Quest19_PAQ610', 'replace': [77, 99]}, # Number of days vigorous work
    {'name': 'Quest19_PAD615', 'replace': [7777	, 9999]}, # Minutes vigorous-intensity work
    {'name': 'Quest19_PAQ640', 'replace': [77	, 99]}, # Number of days walk or bicycle
    {'name': 'Quest19_PAD645', 'replace': [7777	, 9999]}, # Minutes walk/bicycle for transportation
    {'name': 'Quest19_PAQ655', 'replace': [77	, 99]}, # Days vigorous recreational activities
    {'name': 'Quest19_PAD660', 'replace': [7777	, 9999]}, # Minutes vigorous recreational activities
    {'name': 'Quest21_SLD012', 'replace': []}, # "Sleep hours - weekdays or workdays"
    {'name': 'Quest21_SLD013', 'replace': []}, # Sleep hours - weekends
    # Timestamp cols (Need to convert to int first)
    {'name': 'Quest21_SLQ300', 'replace': []}, # "Usual sleep time on weekdays or workdays"
    {'name': 'Quest21_SLQ320', 'replace': []}, # Usual sleep time on weekends
    {'name': 'Quest21_SLQ330', 'replace': []}, # Usual wake time on weekends
]
bool_value_map = {
    'Quest1_ALQ111': '9',
    'Quest2_BPQ020': '9',
    'Quest3_CDQ008': '9',
    'Quest3_CDQ010': '9',
    'Quest7_DIQ010': '9',
    'Quest9_DLQ050': '9',
    'Quest10_ECQ020': '9',
    'Quest11_HIQ011': '9',
    'Quest12_HEQ010': '9',
    'Quest12_HEQ030': '9',
    'Quest14_INQ020': '9',
    'Quest14_INQ012': '9',
    'Quest14_IND235': '99',
    'Quest15_KIQ022': '9',
    'Quest15_KIQ026': '9',
    'Quest17_DPQ020': '9',
    'Quest17_DPQ030': '9',
    'Quest17_DPQ040': '9',
    'Quest18_OCQ210': '9',
    'Quest19_PAQ635': '9',
    'Quest20_PFQ020': '9',
    'Quest20_PFQ061B': '9',
    'Quest20_PFQ061C': '9',
    'Quest20_PFQ061H': '9',
    'Quest22_SMQ020': '9',
    'Quest22_SMQ890': '9',
    'Quest22_SMQ900': '9',
}
special_treatment_cols = [
    'Quest3_CDQ009A',
    'Quest3_CDQ009B',
    'Quest3_CDQ009C',
    'Quest3_CDQ009D',
    'Quest3_CDQ009E',
    'Quest3_CDQ009F',
    'Quest3_CDQ009G',
    'Quest3_CDQ009H',
    'Quest23_SMD470'
]

# ------------------------
# Dropping Columns
# ------------------------

for group in group_to_drop:
  quest = quest[quest.columns.drop(list(quest.filter(regex=group)))]

quest = quest.drop(columns = [col for col in quest.columns if col in cols_to_drop])

# ------------------------
# Converting Timestamp Columns to Continuous
# ------------------------

# HH:MM (hour and minute) to integer --> change to minute representation (00:00 as 0 and 23:59 as 1439)

def timestamp_to_int(x):
  try:
    x = re.search(r"b'(\d+:\d+)'", x).group(1)
    hour = x.split(":")[0]
    minute = x.split(":")[1]
  except:
    return None

  return int(hour)*60 + int(minute)

for col in timestamp_cols:
  for val in col['replace']:
    quest[col['name']] = quest[col['name']].replace(val, None)
  quest[col['name']] = quest[col['name']].apply(timestamp_to_int)

# ------------------------
# Filling Continuous Columns
# ------------------------

quest_mean = quest.copy()

for col in cont_fill_mean:
  for val in col['replace']:
    quest_mean[col['name']] = quest_mean[col['name']].replace(val, None)
  # Option 1 Using Mean by Age
  # quest_mean[col['name']] = quest_mean[col['name']].fillna(quest_mean.groupby('Demog1_RIDAGEYR')[col['name']].transform('mean'))

  # Option 2 Using windowed Mean by Age
  quest_mean[col['name']] = window_mean_by_age(quest_mean[[col['name'], 'Demog1_RIDAGEYR']], col_name=col['name'], window=5)

  # Option 1 Fill the rest using 0
  # quest_mean[col['name']] = quest_mean[col['name']].fillna(0)
  # Option 2 Fill the rest using overall mean
  quest_mean[col['name']] = quest_mean[col['name']].fillna(quest_mean[col['name']].mean())

quest = quest.fillna(quest_mean)

# ------------------------
# Filling Boolean Columns to "Don't Know"
# ------------------------

for col, val in bool_value_map.items():
  quest[col] = quest[col].replace(int(re.sub('9', '7', val)), None)
  quest[col] = quest[col].fillna(9)


# ------------------------
# Filling Special Treatment Columns
# ------------------------

quest['Quest3_CDQ009'] = quest['Quest3_CDQ009A'].notnull() | quest['Quest3_CDQ009B'].notnull() | quest['Quest3_CDQ009C'].notnull() | quest['Quest3_CDQ009D'].notnull() | quest['Quest3_CDQ009E'].notnull() | quest['Quest3_CDQ009F'].notnull() | quest['Quest3_CDQ009G'].notnull() | quest['Quest3_CDQ009H'].notnull()
quest['Quest3_CDQ009'] = quest['Quest3_CDQ009'].replace({True: 1, False: 0}).astype(float)

def family_smoker(x):
  if x in (1,2,3):
    return 1
  return 0

quest['Quest23_SMD470'] = quest['Quest23_SMD470'].apply(family_smoker)

quest = quest.drop(columns = [col for col in quest.columns if col in special_treatment_cols])

model_var = pd.concat([model_var, pd.DataFrame({
    'model_var_name': ['Quest3_CDQ009'],
    'name': ['Questionnaire'],
    'title': ['Cardiovascular Health'],
    'variable': ['CDQ009'],
    'desc': ['Pain In Body (Combined)'],
    'is_used': [True],
    'lifestyle':[False],
    'group': ['Quest3']
    })], ignore_index=True)

# ------------------------
# Making sure Timestamp cols are integer so that it can be reverted to timestamp again
# ------------------------

for col in timestamp_cols:
  quest[col['name']] = quest[col['name']].round(0)

# ------------------------
# Final
# ------------------------

quest = quest[quest.columns.drop(list(quest.filter(regex='Demog')))]

quest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 55 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Quest1_ALQ111    9254 non-null   float64
 1   Quest2_BPQ020    9254 non-null   float64
 2   Quest3_CDQ008    9254 non-null   float64
 3   Quest3_CDQ010    9254 non-null   float64
 4   Quest4_CBD111    9254 non-null   float64
 5   Quest4_CBD121    9254 non-null   float64
 6   Quest6_DED120    9254 non-null   float64
 7   Quest6_DED125    9254 non-null   float64
 8   Quest7_DIQ010    9254 non-null   float64
 9   Quest9_DLQ050    9254 non-null   float64
 10  Quest10_ECQ020   9254 non-null   float64
 11  Quest11_HIQ011   9254 non-null   float64
 12  Quest12_HEQ010   9254 non-null   float64
 13  Quest12_HEQ030   9254 non-null   float64
 14  Quest14_INQ020   9254 non-null   float64
 15  Quest14_INQ012   9254 non-null   float64
 16  Quest14_IND235   9254 non-null   float64
 17  Quest15_K

# Cleaning Dietary Data

In [ ]:
dieta = df_raw[[col for col in df_raw.columns if "Dieta" in col]]
col_to_fill = dieta.columns

dieta = dieta.join(demog)

for col in col_to_fill:
  dieta[col] = dieta[col].fillna(dieta.groupby('Demog1_RIDAGEYR')[col].transform('mean'))

dieta = dieta[dieta.columns.drop(list(dieta.filter(regex='Demog')))]

dieta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Dieta1_DRDINT    9254 non-null   float64
 1   Dieta1_DR1DAY    9254 non-null   float64
 2   Dieta1_DR1TKCAL  9254 non-null   float64
 3   Dieta1_DR1TPROT  9254 non-null   float64
 4   Dieta1_DR1TCARB  9254 non-null   float64
 5   Dieta1_DR1TSUGR  9254 non-null   float64
 6   Dieta1_DR1TFIBE  9254 non-null   float64
 7   Dieta1_DR1TTFAT  9254 non-null   float64
 8   Dieta1_DR1TSFAT  9254 non-null   float64
 9   Dieta1_DR1TMFAT  9254 non-null   float64
 10  Dieta1_DR1TPFAT  9254 non-null   float64
 11  Dieta1_DR1TCHOL  9254 non-null   float64
 12  Dieta1_DR1TCALC  9254 non-null   float64
dtypes: float64(13)
memory usage: 1.2 MB


# Final Transformation

In [ ]:
# Null is because some question are not meant for person below certain ages, but for all respondent with age above 16 have no null values (Except Dietary)

final = demog.copy()
final = final.join(labor)
final = final.join(exam)
final = final.join(quest)
final = final.join(dieta)

final = final.reset_index()

final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9254 entries, 0 to 9253
Data columns (total 92 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SEQN             9254 non-null   float64
 1   Demog1_RIAGENDR  9254 non-null   float64
 2   Demog1_RIDAGEYR  9254 non-null   float64
 3   Demog1_RIDRETH3  9254 non-null   float64
 4   Demog1_DMDMARTL  9254 non-null   float64
 5   Demog1_DMDHHSIZ  9254 non-null   float64
 6   Demog1_DMDFMSIZ  9254 non-null   float64
 7   Demog1_DMDHHSZA  9254 non-null   float64
 8   Demog1_DMDHHSZB  9254 non-null   float64
 9   Demog1_DMDHHSZE  9254 non-null   float64
 10  Demog1_DMDEDUC   9254 non-null   float64
 11  Demog1_INDIN2    9254 non-null   float64
 12  Labor1_LBXTC     9254 non-null   float64
 13  Labor1_LBDTCSI   9254 non-null   float64
 14  Labor2_URXVOL1   9254 non-null   float64
 15  Labor2_URDFLOW1  9254 non-null   float64
 16  Labor2_URDTIME1  9254 non-null   float64
 17  Exami1_BPXPLS 

# Save Data

In [ ]:
# target_path = "Dataset/Data Versioning/Combined_All_V6.csv"

# final.to_csv(os.path.join(drive_path, target_path))

# Variable Mapping V2

In [ ]:
final_var = pd.DataFrame({'model_var_name': final.columns.tolist()})

final_var = pd.merge(final_var, model_var[['model_var_name', 'lifestyle']], on=['model_var_name'], how='outer', indicator=True).query('_merge!="right_only"')
final_var[final_var['_merge'] == "left_only"]

,model_var_name,lifestyle,_merge
0,SEQN,NaN,left_only


In [ ]:
model_var

,model_var_name,name,title,variable,desc,is_used,lifestyle,group
0,Demog1_RIAGENDR,Demographics,Demographic Variables and Sample Weights,RIAGENDR,Gender,True,False,Demog1
1,Demog1_RIDAGEYR,Demographics,Demographic Variables and Sample Weights,RIDAGEYR,Age In Years At Screening,True,False,Demog1
2,Demog1_RIDRETH3,Demographics,Demographic Variables and Sample Weights,RIDRETH3,Race/Hispanic Origin W/ Nh Asian,True,False,Demog1
3,Demog1_DMDEDUC3,Demographics,Demographic Variables and Sample Weights,DMDEDUC3,Education Level - Children/Youth 6-19,True,False,Demog1
4,Demog1_DMDEDUC2,Demographics,Demographic Variables and Sample Weights,DMDEDUC2,Education Level - Adults 20+,True,False,Demog1
...,...,...,...,...,...,...,...,...
148,Demog1_DMDEDUC,Demographic,Demographic Variables and Sample Weights,DMDEDUC,Education Level all age,True,False,Demog1
149,Demog1_INDIN2,Demographic,Demographic Variables and Sample Weights,INDIN2,Average income from household income & family ...,True,False,Demog1
150,Exami1_SysPulse,Examination,Blood Pressure,SysPulse,Systolic Pulse Combined,True,False,Exami1
151,Exami1_DiaPulse,Examination,Blood Pressure,DiaPulse,Diastolic Pulse Combined,True,False,Exami1


In [ ]:
# model_var.to_excel(drive_path + "/Dataset/Variable Mapping V2.xlsx", index=False)